# Orizon: Correlações entre Exames

Vamos entender como os serviços se relacionam entre si computando a frequencia relativa à cada um deles por ID conta. Por exemplo, para um serviço X, selecionamos todos os id_contas em que ele aparece, e então computamos a frequencia em que os outros serivços aparecem junto deste serviço. 

In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from IPython.display import display

Lendo os dados para a dataframe:

In [2]:
df = pd.read_csv("../orizon/data_30726034.csv", encoding = "ISO-8859-1")
master = "'30726034'"

/home/raven/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,'categoria','id_cliente','cliente','id_lote','id_conta','guia_prestador','id_item','id_prestador','prestador','cnpj',...,'qtde','valor','origem','cnpj_cpf_codnaoperadora_executante','data','ano_mes',Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50
0,'Seguradora Especializada em Saude','48','BRADESCO SAÃDE','42704626','1135485394','3280780','486879189','46069','CLINICA ORTOPEDICA PAULISTA LTDA','03140417000148',...,'ARTROPLASTIA TOTAL DE JOELHO COM IMPLANTES - ...,'1.0','2000.68','0000282693','0000282693','2016-01-02',NaN,NaN,NaN,NaN
1,'Seguradora Especializada em Saude','48','BRADESCO SAÃDE','42704626','1135485394','3280780','486879189','46069','CLINICA ORTOPEDICA PAULISTA LTDA','03140417000148',...,'ARTROPLASTIA TOTAL DE JOELHO COM IMPLANTES - ...,'1.0','2000.68','0000282693','0000282693','2016-01-02',NaN,NaN,NaN,NaN
2,'Seguradora Especializada em Saude','48','BRADESCO SAÃDE','42704626','1135485394','3280780','486879189','46069','CLINICA ORTOPEDICA PAULISTA LTDA','03140417000148',...,'ARTROPLASTIA TOTAL DE JOELHO COM IMPLANTES - ...,'1.0','2000.68','0000282693','0000282693','2016-01-02',NaN,NaN,NaN,NaN
3,'Seguradora Especializada em Saude','48','BRADESCO SAÃDE','42704626','1135485394','3280780','486879191','46069','CLINICA ORTOPEDICA PAULISTA LTDA','03140417000148',...,'ARTROPLASTIA TOTAL DE JOELHO COM IMPLANTES - ...,'1.0','400.14','0000282693','0000282693','2016-01-02',NaN,NaN,NaN,NaN
4,'Seguradora Especializada em Saude','48','BRADESCO SAÃDE','42704626','1135485394','3280780','486879191','46069','CLINICA ORTOPEDICA PAULISTA LTDA','03140417000148',...,'ARTROPLASTIA TOTAL DE JOELHO COM IMPLANTES - ...,'1.0','400.14','0000282693','0000282693','2016-01-02',NaN,NaN,NaN,NaN


In [4]:
df['\'data\''].loc[df['\'servico\''].str.contains(r'^.{4}(?!\d+).*')] = 'Deslocado'
df['\'tipo_item\''].loc[df['\'data\''] == 'Deslocado'] = df['\'desc_tipoalta\''].loc[df['\'data\''] == 'Deslocado']
df['\'servico\''].loc[df['\'data\''] == 'Deslocado'] = df['\'descricao_despesa\''].loc[df['\'data\''] == 'Deslocado']
df['\'descricao_despesa\''].loc[df['\'data\''] == 'Deslocado'] = df['\'qtde\''].loc[df['\'data\''] == 'Deslocado']
df['\'qtde\''].loc[df['\'data\''] == 'Deslocado'] = df['\'valor\''].loc[df['\'data\''] == 'Deslocado']
df['\'valor\''].loc[df['\'data\''] == 'Deslocado'] = df['\'origem\''].loc[df['\'data\''] == 'Deslocado']

Selecionando apenas os campos relevantes para esta análise:

In [5]:
selected_df = pd.concat([df["'id_conta'"], df["'servico'"], df["'descricao_despesa'"], df["'tipo_item'"]], axis=1)

Removendo as aspas das labels para facilitar o trabalho:

In [6]:
selected_df.columns = ["id_conta", "servico", "descricao", "tipo_item"]

Vamos trabalhar apenas com os id_conta atrelados à mais de um serviço:

In [7]:
count_df = selected_df.groupby("id_conta").count()

In [8]:
def get_count(acc_id):
    return count_df.lookup([acc_id], ["servico"])[0]

In [9]:
selected_df["count"] = selected_df["id_conta"].map(get_count)

In [10]:
selected_df = selected_df[selected_df["count"] > 1].drop("count", axis=1)

Algumas observações possuem apenas " no campo serviço, vamos filtra-las também:

In [11]:
selected_df = selected_df[selected_df["servico"] != '"']
selected_df = selected_df[selected_df["servico"] != "''"]

In [12]:
selected_df.head()

,id_conta,servico,descricao,tipo_item
19,'1135853444','30726239','Realinhamentos do aparelho extensor - tratame...,'PROCEDIMENTO'
20,'1135853444','30726034','Artroplastia total de joelho com implantes - ...,'PROCEDIMENTO'
21,'1135853444','30726042','Artrotomia - tratamento cirÃºrgico','PROCEDIMENTO'
22,'1135853446','30726042','Artrotomia - tratamento cirÃºrgico','PROCEDIMENTO'
23,'1135853446','30726239','Realinhamentos do aparelho extensor - tratame...,'PROCEDIMENTO'


O resultado que estamos buscando é uma tabela que mostre, para cada serviço, os serviços que aparecem com mais frequencia junto dele. Para isso, começamos definindo uma função para selecionar apenas os id_conta que contem um dado serviço:

In [13]:
def select_ids_containing_service(service):
    acc_ids = selected_df[selected_df["servico"] == service]["id_conta"].values
    return selected_df.loc[selected_df["id_conta"].isin(acc_ids)]

Agora, definimos uma função para computar as frequencias de cada serviço dentro das dataframes selecionadas pela função acima:

In [14]:
def get_percentage(service, service_df):
    unique_acc_ids = len(service_df["id_conta"].unique())
    count_num = len(service_df[service_df["servico"] == service]["id_conta"].unique())
    return (float(count_num) / unique_acc_ids) * 100.0

O mesmo conceito do select_ids_containing_service, só que essa função também funciona para um conjunto de serviços, retornando o número total de contas que o grupo aparece:

In [15]:
def support(service):
    acc_ids = selected_df[selected_df["servico"] == service[0]]["id_conta"].values
    for i in range(1,len(service)):
        acc_ids = selected_df[(selected_df["servico"] == service[i]) & (selected_df["id_conta"].isin(acc_ids))]["id_conta"].values
    return selected_df.loc[selected_df["id_conta"].isin(acc_ids)]["id_conta"].unique().shape[0]

Essa função servirá para o algoritmo de associação do eclat, ele elimina os grupos de serviços que contém todos os serviços de um lista de elementos.

In [16]:
def drop_comb(elements,grupo):
    return len(list(filter(lambda x: x in elements, grupo))) != len(elements)

Função que retorna uma lista com todas as combinações possíveis dados os elemeentos de uma lista e o total de elementos por grupo, ela também só retornará apenas os grupos que não estão contidos em uma lista de elementos, utilizando a função drop_comb:

In [17]:
def combinations_test(iterable, r,drops):
    # combinations('ABCD', 2) --> AB AC AD BC BD CD
    # combinations(range(4), 3) --> 012 013 023 123
    pool = tuple(iterable)
    n = len(pool)
    if r > n:
        return
    indices = list(range(r))
    comb = tuple(pool[i] for i in indices)
    i=0
    for drop in drops:
        if drop_comb(drop,comb)==False:
            break;
        i=i+1
        if i==len(drops):
            yield comb
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return
        indices[i] += 1
        for j in range(i+1, r):
            indices[j] = indices[j-1] + 1
        comb = tuple(pool[i] for i in indices)
        i=0
        for drop in drops:
            if drop_comb(drop,comb)==False:
                break;
            i=i+1
            if i==len(drops):
                yield comb

Função com base no algoritmo eclat, que agrupará os serviços de acordo com o seu suporte(vezes que aparece na conta):

In [18]:
def eclat(lista,i=1,ids_drop=[],min_sup=10):
    serv_list=[]
    ids = []
    new_ids_drop = []
    if(i==1):
        for service in lista:
            supp = support([service])
            if(supp>min_sup):
                serv_list.append(supp)
                ids.append(service)
        return ids,serv_list,ids_drop
    else:
        if len(ids_drop)>0:
            ids_2 = list(combinations_test(lista,i,ids_drop))
        else:
            ids_2 = list(combinations(lista,i))
        
#         if len(ids_drop)>0:
#             for drops in ids_drop:
#                 ids_2 = list(filter(lambda x: drop_comb(elements=drops,lista=x),ids_2))
        for service in ids_2:
            supp = support(service)
            if(supp>min_sup):
                serv_list.append(supp)
                ids.append(service)
            else:
                new_ids_drop.append(service)
        return ids,serv_list,new_ids_drop
    

Retornando o número de contas equivalentes a 30% do total de contas, para utilizar como mínimo suporte

In [34]:
min_sup = int(0.3*support([master]))

Iteração final:

In [35]:
lista = selected_df['servico'].unique()
ids, serv_list, ids_drop = eclat(lista,min_sup=min_sup)
i=2
final_id = ids
final_serv = serv_list
new_serv = [1,1]
while True:
    if(len(new_serv)<2):
        break
    else:
        print(i)
        new_ids, new_serv, ids_drop = eclat(lista=ids,ids_drop=ids_drop,i=i,min_sup=min_sup)
        final_id = final_id + new_ids
        final_serv = final_serv + new_serv
        i = i + 1
        ids = []
        for x in new_ids:
            for y in x:
                if y not in ids:
                    ids.append(y)


    

2
3
4
5
6
7
8


Passando o resultado para um DataFrame e na ordem do maior para o menor:

In [36]:
final_result = pd.DataFrame(index=final_id,data=final_serv,columns=['Suporte']).sort_values('Suporte',ascending=False)
final_result.Suporte = final_result.Suporte * (100.0/917)
final_result

,Suporte
'30726034',100.000000
'40304361',52.998909
"('30726034', '40304361')",52.998909
'90196031',49.509269
"('30726034', '90196031')",49.509269
'40301630',48.200654
"('30726034', '40301630')",48.200654
"('40304361', '40301630')",47.873501
"('30726034', '40304361', '40301630')",47.873501
"('30726034', '40302580')",47.328244


Vamos definir como raio X o grupo que possuir a maior quantidade de serviços que, em conjunto, possuem maior suporte:

In [121]:
def get_len(x):
    if type(x) == tuple:
        return len(x)
    return 1

max_len = max([get_len(x) for x in list(final_result.index)]) # acha o tamanho dos maiores grupos
max_len_groups = [x for x in list(final_result.index) if get_len(x) == max_len] # pega grupos com esse tamanho
xray = list(final_result.loc[max_len_groups].sort_values('Suporte', ascending=False).head(1).index[0]) # pega o com maximo suporte

In [122]:
xray

[u"'30726034'",
 u"'40304361'",
 u"'40302237'",
 u"'40302318'",
 u"'40302423'",
 u"'40302580'",
 u"'40301630'"]

Agora, com o raio x definido, construimos a dataframe final, com colunas servico, descricao, tipo de item, valor medio e quantidade media.

In [93]:
def get_mean_value(service):
    values = []
    for item in df[df["'servico'"] == service]["'valor'"].values:
        try:
            values.append(float(item[1:-1]))
        except ValueError: # some values are wrong, ignore them
            pass
    return sum(values) / len(values)

In [94]:
def get_mean_quantity(service):
    values = []
    for item in df[df["'servico'"] == service]["'qtde'"].values:
        try:
            values.append(float(item[1:-1]))
        except ValueError: # some values are wrong, ignore them
            pass
    return sum(values) / len(values)

In [54]:
def get_description(service):
    return df[df["'servico'"] == service]["'descricao_despesa'"].values[0]

In [73]:
def get_item_type(service):
    return df[df["'servico'"] == service]["'tipo_item'"].values[0]

In [80]:
def get_individual_support(service):
    return final_result.loc[service].values[0]

In [123]:
mean_vals = [get_mean_value(service) for service in xray]
mean_qtt = [get_mean_quantity(service) for service in xray]
item_types = [get_item_type(service) for service in xray]
descriptions = [get_description(service) for service in xray]
support_vals = [get_individual_support(service) for service in xray]
xray_dict = {"Servico": xray, "Descricao":descriptions, "Tipo de Item":item_types, "Quantidade média": mean_qtt, 
             "Valor médio": mean_vals, "Suporte":support_vals}
xray_df = pd.DataFrame(xray_dict)

In [124]:
xray_df

,Descricao,Quantidade média,Servico,Suporte,Tipo de Item,Valor médio
0,'ARTROPLASTIA TOTAL DE JOELHO COM IMPLANTES TR...,1.004676,'30726034',100.000000,'PROCEDIMENTO',644.608995
1,'HEMOGRAMA COM CONTAGEM DE PLAQUETAS OU FRAES ...,1.074984,'40304361',52.998909,'PROCEDIMENTO',5.859845
2,'MAGNSIO',1.082990,'40302237',35.768811,'PROCEDIMENTO',5.389965
3,'POTSSIO',1.105403,'40302318',46.892039,'PROCEDIMENTO',4.930065
4,'SDIO',1.099345,'40302423',46.782988,'PROCEDIMENTO',4.882831
5,'URIA',1.096811,'40302580',47.328244,'PROCEDIMENTO',4.887392
6,'CREATININA',1.094479,'40301630',48.200654,'PROCEDIMENTO',5.039064
